# Mobile Get Input Notebook - Phase 0: Setup

**Phase 0**: Initialize environment, configure parameters, and prepare elevation data.

This notebook sets up everything needed for the subsequent phases:
- Phase 1: Data Preparation (download/cache land cover)
- Phase 2: Batch Point Generation
- Phase 3: Batch Data Extraction
- Phase 4: Post-processing & Export

## Workflow
1. **Imports**: Load required libraries and modules
2. **Path Setup**: Detect project root and create data directories
3. **Configuration**: Define transmitter location and processing parameters
4. **Transmitter**: Create transmitter object from configuration
5. **Elevation Seed**: Pre-download elevation data (one-time operation)

## Output
- Ready-to-use environment with CONFIG, tx, and elevation data cached
- All subsequent phases can reuse this setup

## Imports

In [6]:
import os
import math
import sys
import time
from pathlib import Path

# Find and add project root to path FIRST (before importing config_sentinel_hub)
possible_roots = [
    Path.cwd(),
    Path.cwd().parent,
    Path.cwd().parent.parent,
]

project_root = None
for root in possible_roots:
    if (root / 'src' / 'mst_gis').exists() or (root / 'config_sentinel_hub.py').exists():
        project_root = root
        break

if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Now we can import config_sentinel_hub
import requests
from config_sentinel_hub import (
    SH_CLIENT_ID, SH_CLIENT_SECRET,
    TOKEN_URL, PROCESS_URL, COLLECTION_ID,
)

# Data processing imports
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, box
from dataclasses import dataclass
from typing import Iterable, Union, List, Tuple

# GIS and raster imports
import rasterio
import rasterio.transform
from rasterio.transform import rowcol
from rasterio.io import MemoryFile
import elevation

# Additional imports for all phases
import ast
import csv
import json
import matplotlib.pyplot as plt

# Define all data paths (used by later cells)
profiles_dir = project_root / 'data' / 'input' / 'profiles'
api_data_dir = project_root / 'data' / 'intermediate' / 'api_data'
workflow_dir = project_root / 'data' / 'intermediate' / 'workflow'
reference_dir = project_root / 'data' / 'input' / 'reference'
output_dir = project_root / 'data' / 'output' / 'spreadsheets'
geojson_dir = project_root / 'data' / 'output' / 'geojson'
# Override elevation cache to use project directory (not system cache)
cache_dir = project_root / 'data' / 'intermediate' / 'elevation_cache'
cache_dir.mkdir(parents=True, exist_ok=True)
elevation.CACHE_DIR = str(cache_dir)
vrt_path = Path(cache_dir) / 'SRTM1' / 'SRTM1.vrt'
zones_geojson_path = reference_dir / 'zones_map_BR.json'

print("✓ All imports successful")

✓ All imports successful


## Path Setup

In [7]:
# Create all required data directories
profiles_dir.mkdir(parents=True, exist_ok=True)
api_data_dir.mkdir(parents=True, exist_ok=True)
workflow_dir.mkdir(parents=True, exist_ok=True)
reference_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)
geojson_dir.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print('✓ All data directories created')
print(f'  profiles: {profiles_dir}')
print(f'  api_data: {api_data_dir}')
print(f'  reference: {reference_dir}')
print(f'  output: {output_dir}')

Project root: /Users/oz/Documents/mst_gis
✓ All data directories created
  profiles: /Users/oz/Documents/mst_gis/data/input/profiles
  api_data: /Users/oz/Documents/mst_gis/data/intermediate/api_data
  reference: /Users/oz/Documents/mst_gis/data/input/reference
  output: /Users/oz/Documents/mst_gis/data/output/spreadsheets


## Configuration

In [8]:
# ============================================================================
# CONFIGURATION - Edit these parameters for your simulation
# ============================================================================

CONFIG = {
    'TRANSMITTER': {
        'tx_id': 'TX_0001',
        'longitude': -13.40694,
        'latitude': 9.345,
        'antenna_height_tx': 57,
        'antenna_height_rx': 10,
    },
    'P1812': {
        'frequency_ghz': 0.9,
        'time_percentage': 50,
        'polarization': 1,
    },
    'RECEIVER_GENERATION': {
        'max_distance_km': 11,
        'azimuth_step': 10,
        'distance_step': 0.03,
        'sampling_resolution': 30,
    },
    'SENTINEL_HUB': {
        'buffer_m': 11000,
        'chip_px': 734,
        'year': 2020,
    },
    'LCM10_TO_CT': {
        100: 1, 80: 2, 30: 2, 40: 2, 70: 2, 110: 2, 254: 2,
        20: 3, 50: 3, 10: 4, 60: 4, 90: 4,
    },
    'CT_TO_R': {1: 0, 2: 0, 3: 10, 4: 15, 5: 20},
}

# Derived values
tx_lon = CONFIG['TRANSMITTER']['longitude']
tx_lat = CONFIG['TRANSMITTER']['latitude']
max_distance_km = CONFIG['RECEIVER_GENERATION']['max_distance_km']
n_points = int(max_distance_km * 1000 / CONFIG['RECEIVER_GENERATION']['sampling_resolution'])
azimuths = list(range(0, 360, CONFIG['RECEIVER_GENERATION']['azimuth_step']))
distances = np.arange(
    CONFIG['RECEIVER_GENERATION']['distance_step'],
    max_distance_km + CONFIG['RECEIVER_GENERATION']['distance_step'],
    CONFIG['RECEIVER_GENERATION']['distance_step']
)

print(f"Transmitter: ({tx_lat}, {tx_lon})")
print(f"Azimuths: {len(azimuths)} | Profile points: {n_points}")
print(f"Frequency: {CONFIG['P1812']['frequency_ghz']} GHz | Polarization: {CONFIG['P1812']['polarization']}")

Transmitter: (9.345, -13.40694)
Azimuths: 36 | Profile points: 366
Frequency: 0.9 GHz | Polarization: 1


## Transmitter Definition

In [9]:
@dataclass
class Transmitter:
    tx_id: str
    lon: float
    lat: float
    htg: float
    f: float
    pol: int
    p: float
    hrg: float

tx = Transmitter(
    tx_id=CONFIG['TRANSMITTER']['tx_id'],
    lon=CONFIG['TRANSMITTER']['longitude'],
    lat=CONFIG['TRANSMITTER']['latitude'],
    htg=CONFIG['TRANSMITTER']['antenna_height_tx'],
    f=CONFIG['P1812']['frequency_ghz'],
    pol=CONFIG['P1812']['polarization'],
    p=CONFIG['P1812']['time_percentage'],
    hrg=CONFIG['TRANSMITTER']['antenna_height_rx'],
)
print(f"\n✓ Transmitter created:")
print(f"  {tx}")


✓ Transmitter created:
  Transmitter(tx_id='TX_0001', lon=-13.40694, lat=9.345, htg=57, f=0.9, pol=1, p=50, hrg=10)


## Elevation Data Preparation

Seed elevation data once (required for all subsequent phases).

In [10]:
# KEY OPTIMIZATION: Seed elevation ONCE before any phase
# This downloads and caches the DEM tiles needed for the analysis area
print("\nSeeding elevation data...")
seed_start = time.time()

try:
    # Set bounds around transmitter (0.1 deg ~ 11 km)
    bounds = [tx_lon - 0.1, tx_lat - 0.1, tx_lon + 0.1, tx_lat + 0.1]
    
    # Download SRTM1 tiles to local cache
    elevation.seed(
        cache_dir=str(cache_dir),
        bounds=bounds,
        product='SRTM1',
        max_download_tiles=9
    )
    
    seed_time = time.time() - seed_start
    print(f"✓ Elevation data ready ({seed_time:.2f}s)")
    print(f"  Cache location: {cache_dir}")
    
    # Verify files exist
    cache_files = list(cache_dir.rglob('*.tif'))
    if cache_files:
        print(f"  Downloaded {len(cache_files)} DEM tile(s)")
        for f in sorted(cache_files)[:3]:
            print(f"    - {f.relative_to(cache_dir.parent.parent)}")
    else:
        print(f"  Warning: No DEM tiles in cache after seeding")
except Exception as e:
    import traceback
    print(f"Error: {e}")
    traceback.print_exc()
    seed_time = 0

print("\n" + "="*60)
print("PHASE 0 COMPLETE: Setup ready for subsequent phases")
print("="*60)
print("\nNext steps:")
print("  1. Run Phase 1 to download/cache land cover data")
print("  2. Run Phase 2 to generate all receiver points")
print("  3. Run Phase 3 to extract elevation and land cover")
print("  4. Run Phase 4 to format and export profiles")


Seeding elevation data...
make: Nothing to be done for `download'.
make: Nothing to be done for `all'.
✓ Elevation data ready (0.05s)
  Cache location: /Users/oz/Documents/mst_gis/data/intermediate/elevation_cache

PHASE 0 COMPLETE: Setup ready for subsequent phases

Next steps:
  1. Run Phase 1 to download/cache land cover data
  2. Run Phase 2 to generate all receiver points
  3. Run Phase 3 to extract elevation and land cover
  4. Run Phase 4 to format and export profiles


## Summary

**Phase 0 Complete**:
- ✓ All imports loaded
- ✓ Project paths created
- ✓ Configuration defined (CONFIG dictionary)
- ✓ Transmitter object created
- ✓ Elevation data cached locally

**Output**: Ready-to-use environment with all setup complete.

**Duration**: ~{seed_time:.2f}s (mainly for elevation.seed())

**Next**: Proceed to Phase 1 (Data Preparation)